## Training Yolo for custom Object Detection

In [1]:
# Checking Gpu
!nvidia-smi

Thu Jul 30 08:12:18 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# connecting Google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
# Cloning the DARKNET!
!git clone https://github.com/AlexeyAB/darknet
# Also my repo where I have helper scripts!
!git clone https://github.com/luckyCasualGuy/Scripts.git

Cloning into 'darknet'...
remote: Enumerating objects: 14086, done.
remote: Total 14086 (delta 0), reused 0 (delta 0), pack-reused 14086
Receiving objects: 100% (14086/14086), 12.72 MiB | 11.30 MiB/s, done.
Resolving deltas: 100% (9587/9587), done.
Cloning into 'Scripts'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 35 (delta 9), reused 27 (delta 7), pack-reused 0
Unpacking objects: 100% (35/35), done.


**Configuring DARKNET**:  
setting:  
`OPENCV ` -> True  
`GPU` -> True  
`CUDNN` -> True  

In [ ]:
# configuring & making DARKNET!
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

##We will stay in DARKNET dir as final training will be done here!

In [5]:
# Making some required path shortcuts
!ln -s 'cfg/yolov3-tiny.cfg' original_model
!cp original_model 'cfg/yolov3-tiny-model.cfg'
!ln -s 'cfg/yolov3-tiny-model.cfg' original_model_copy
# some shortcuts to my drive
!ln -s '../drive/My Drive' my_drive
!ln -s 'my_drive/OD/dataset' my_data
!ln -s 'my_drive/OD/weights' weights
!ln -s 'my_drive/OD/output' output
# path to my script repo
!ln -s '../Scripts' my_repo
!ln -s 'my_repo/Object detection/' my_scripts
# path to root dir
!ln -s '../' root

These are the appropriate changes made!

In [6]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3-tiny-model.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3-tiny-model.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3-tiny-model.cfg
!sed -i '135 s@classes=80@classes=2@' cfg/yolov3-tiny-model.cfg
!sed -i '177 s@classes=80@classes=2@' cfg/yolov3-tiny-model.cfg
!sed -i '127 s@filters=255@filters=21@' cfg/yolov3-tiny-model.cfg
!sed -i '171 s@filters=255@filters=21@' cfg/yolov3-tiny-model.cfg

In [ ]:
# unzip images in yolo form in the root folder.
!unzip my_data/ok_wait.zip -d root

In [9]:
# Running script to get obj.data
!python my_scripts/make_obj_3_6.py -d root/ok_wait -s root/obj -p 90 -o output

Done! :)


In [ ]:
# Start the training
!./darknet detector train root/obj/obj.data original_model_copy weights/yolov3-tiny.conv.11 -dont_show